# Phase 1: Preprocessing with Regular Expressions

In [1]:
import subprocess
subprocess.run(['git', 'pull'])

Already up to date.


CompletedProcess(args=['git', 'pull'], returncode=0)

In [2]:
subprocess.run(['git', 'add', 'Haris_Saif.ipynb'])
subprocess.run(['git', 'commit', '-m', 'regex'])

On branch main
Your branch is ahead of 'origin/main' by 24 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Kyle_Choi.ipynb
	deleted:    memos_cleaned.csv

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	memos_P1.csv
	sample.csv

no changes added to commit (use "git add" and/or "git commit -a")


CompletedProcess(args=['git', 'commit', '-m', 'regex'], returncode=1)

## 1. Load Dataset


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import time
import sys
import re

In [4]:
df_in = pd.read_csv("memos.csv")
df = df_in#.sample(100_000).copy()
row_count = df.size
row_count

528766

In [5]:
df['memo'].sample(25).sort_values().reset_index(drop=True)

0     #4U5K2PCLOWVG AMAZON.COM*1U5BW9IR1 SEATTLE WA ...
1     CAPTAIN D'S XXXX 01-26 LITHONIAGA XXXX DEBIT C...
2     CHECKCARD XXXX SQ *CORNER SPORTS North Miami B...
3     DEBIT CARD PURCHASE XXXXXXXXX LA GRANJA PARRIL...
4                                DOLLAR TR XXXX E FLAMI
5     GOOGLE *King g.co/helppay# CA                0...
6                      HOMETOWN PIZZA 001 - E 502-XXXXX
7              INFINITY SMOKE SHOP MORRISVILLE PA 07/25
8     Microsoft*Subscrip 10-28 msbill.info WA XXXX D...
9     POS Debit - Visa Check Card XXXX - SUSHI KING ...
10    POS Debit - Visa Check Card XXXX - TIFFANY CO ...
11    PURCHASE AUTHORIZED ON 01/10 MANOLO RESTAURANT...
12    PURCHASE AUTHORIZED ON 01/30 UBER TRIP HELP.UB...
13    PURCHASE AUTHORIZED ON 02/14 CHICK-FIL-A #XXXX...
14    PURCHASE AUTHORIZED ON 04/23 DD'S DISCOUNTS #5...
15    PURCHASE AUTHORIZED ON 06/18 ALBERTACOS MEXICA...
16    PURCHASE AUTHORIZED ON 07/06 CUB FOODS WSP WES...
17    PURCHASE AUTHORIZED ON 09/24 ROSS STORES #

## 2. Define Regex Rules

In [6]:
STATE_LIST = [
    "AL", "AK", "AZ", "AR", "CA", 
    "(?<!\.)CO(?!['`])", # Negative lookbehind/ahead for CO (e.g., not .CO or COSTCO)
    "CT", "DC", "DE", "FL", "GA", "HI", "IA", 
    "ID", "IL", 
    "IN(?!\\s+N\\s+OUT\\s+BURGER)", # Negative lookahead for IN (not IN N OUT BURGER)
    "KS", "KY",
    "(?<!['`])LA(?!\\s+HACIENDA|\\s+FITNESS|\\s+LA'S|['`])", # Negative lookaheads for LA
    "MA", "MD", 
    "ME(?!\\s+DIA)", # Negative lookahead for ME (not ME DIA)
    "MI", "MN", "MO(?!['`])", 
    "MS", "MT", "NC", "ND", "NE", "NH", "NJ", "NM", "NV", "NY", 
    "OH", "OK", "OR", 
    "PA(?!['`])", 
    "RI", "SC", "SD", "TN", "TX", "UT", "VA", "VT", "WA", 
    "WI", "WV", "WY"
]
STATE_REGEX = r"\b(" + "|".join(STATE_LIST) + r")\b"

In [ ]:
# In[6]:


STATE_LIST = [ 
    "AL", "AK", "AZ", "AR", "CA", 
    "(?<!\.)CO(?!['`])", # Negative lookbehind/ahead for CO (e.g., not .CO or COSTCO) 
    "CT", "DC", "DE", "FL", "GA", "HI", "IA", 
    "ID", "IL", 
    "IN(?!\\s+N\\s+OUT\\s+BURGER)", # Negative lookahead for IN (not IN N OUT BURGER) 
    "KS", "KY", 
    "(?<!['`])LA(?!\\s+HACIENDA|\\s+FITNESS|\\s+LA'S|['`])", # Negative lookaheads for LA 
    "MA", "MD", 
    "ME(?!\\s+DIA)", # Negative lookahead for ME (not ME DIA) 
    "MI", "MN", "MO(?!['`])", 
    "MS", "MT", "NC", "ND", "NE", "NH", "NJ", "NM", "NV", "NY", 
    "OH", "OK", "OR", 
    "PA(?!['`])", 
    "RI", "SC", "SD", "TN", "TX", "UT", "VA", "VT", "WA", 
    "WI", "WV", "WY" 
] 
STATE_REGEX = r"\b(" + "|".join(STATE_LIST) + r")\b" 

# NEW: City list from 1-grams and user example
CITY_LIST = [
    "SAN", "MIAMI", "DIEGO", "PHOENIX", "SEATTLE", "HOUSTON", "SANTA", "ORLANDO", 
    "CHICAGO", "LAS", "ATLANTA", "LOS", "MESA", "VEGAS", "CHARLOTTE", "TAMPA", 
    "GREENVILLE", "BROOKLYN", "DENVER", "ANGELES", "ANTONIO", "MEMPHIS", "YORK",
    "RICHMOND", "BEACH", "PALM", "FORT", "ST", "LAKE", "WEST", "DES", "PARK",
    "HILL", "NORTH", "SPRING", "CREEK", "SAINT", "RIVER", "SOUTH", "MYERS"
]
CITY_REGEX = r"\b(" + "|".join(CITY_LIST) + r")\b"

In [7]:
# From 1-gram
NOISE_WORDS = [
    "DBT", "PURCH", "TRANSACTION", "PMT", "PMTS", "HTTPSWWW", "WWW", "CONSUMER", 
    "CKCD", "CRD", "PUR", "PIN", "SIG", "LLC", "SIGNATURE", "WEB", "PAYMENT",
    "ACH", "DEB", "INTL", "RECURRING", "DIGIT", "ONLINE", "PROTECTION", "VSA",
    "800", "888", "WITHDRAWAL", "INDN", "STORE", "STORES", "RESTAURANT", "SUPER",
    "MARKET", "MART", "FOOD", "FOODS", "CAFE", "SHOP", "BAR", "GRILL", "HOUSE",
    "CLUB", "COFFEE", "WINE", "LIQUOR", "LIQUORS", "BEAUTY", "DELI", "SMOKE",
    "GROCERY", "NAILS", "STOP", "BUSINESS", "PARKING", "SPA", "PET", "GARDEN",
    # Added from 1-grams/user example
    "CENTER", "MOBILE", "VENDING", "DIGITAL", "EXPRESS", "FIL", "NEW"
]
NOISE_WORDS_REGEX = r"\b(" + "|".join(NOISE_WORDS) + r")\b"

In [8]:
REGEX_PRE = [ 
    # === 0) Normalize spaces first === 
    (r"\u00A0", " "), # Replace non-breaking space with regular space 
    (r"\s{2,}", " "), # Collapse multiple spaces into one 

    # === 1) “Authorized / Recurring” headers === 
    (r"\b(?:RECURRING\s+)?PAYMENT\s+AUTHORIZED\s+ON(?:\s+\d{2}[/-]\d{2,4})?\b", " "), 
    (r"\b(?:P?URCHASE\s+)?AUTHORIZED\s+ON(?:\s+\d{2}[/-]\d{2,4})?\b", " "), 
    (r"\bAUTHORIZED\s+ON\s+\d{2}[/-]\d{2,4}\b", " "), 
    (r"\bRECURRING\s+PYMT\b", " "), 

    # === 2) Card & mask boilerplate === 
    (r"\b(?:VISA|MASTERCARD|AMEX|DISCOVER)\s+CHECK\s+CARD\b", " "), 
    (r"\bCHECK\s*CARD\b(?:\s*X+)?", " "), 
    (r"\bCARD(?:\s+ENDING\s+IN)?\s*X{4}\b", " "), 
    (r"\bDEBIT\s+CARD\s+DEBIT\s*/", " "), 
    (r"\b(?:DEBIT|CREDIT)\s+CARD\s+(?:PURCHASE|DEBIT|AUTH(?:ORIZATION)?)\b", " "), 
    (r"\b(?:WITHDRAWAL|POS)\s*#", " "), 
    (r"\bWITHDRAWAL\s+DEBIT\s+CHIP\b", " "), 
    (r"\bPOS\s+PUR-\s*(?:\*+)?", " "), 
    (r"\bAUTH\s*#\s*-?", " "), 
    (r"\bCK\s*X+\b", " "), 
    (r"\bPOS\s+(?:PURCHASE|WITHDRAWAL|DEBIT)\b", " "), 
    (r"\b(?:DDA\s+)?PIN\s+POS\s+PUR\b", " "), 
    (r"\bCDX{4,}\b", " "), 
    (r"\bF[XF]{4,}\b", " "), # Handle FXXXXX, FXXXX
    (r"X{4,}", " "), # Remove generic masked numbers 
    (r"\b[SP]X{6,}\b", " "), 
    (r"\bDEBIT\s+(?:CARD|CRD)\b", " "), 
    (r"\bDEBIT\s+PURCHASE\b", " "), 
    (r"\bPOS\s+SIGNATURE\b", " "), 
    (r"\b(?:VISA|MASTERCARD|AMEX|DISCOVER|CARD|DATE|MCC)\b", " "), # Remove common card-related keywords 
    (r"^\s*PURCHASE\b", " "), # Remove "PURCHASE" if at start 
    (r"^\s*REC\s+POS\b", " "), 
    (r"^\s*RECURRING\b", " "), 

    # === 2.5) Prefix Normalization === 
    (r"\b(DNH)(?=[A-Z]{2,})", r"\1 "), # Fix "DNHGODADDYCOM" -> "DNH GODADDYCOM" 
    (r"\bDD\s*(?:[\\/]\s*)?BR\b", "DDBR"), # Combine DD/BR or DD BR -> DDBR 

    # === 3) State + mask tails === 
    (r"\b[A-Z]{2}\s+[SP]?X{6,}\s+CARD\s+X{4}\b", " "), 
    (r"\b[A-Z]{2}\s+[SP]?X{6,}\b", " "), 

    # === 4) Dates/times === 
    (r"\b#?\d{2}[/-]\d{2}(?:[/-]\d{2,4})?\b", " "), # Dates like 10/23, 10/23/2025 
    (r"\b\d{1,2}\s+\d{2}\s+\d{2}\s*(?:AM|PM)\b", " "), # 10 23 25 PM 
    (r"\b\d{1,2}:\d{2}(?::\d{2})?\s*(?:AM|PM)\b", " "), # Times like 10:23 AM 

    # === 5) Merchant-terminal boilerplate === 
    (r"\bMERCHANT\s+PURCHASE\s+TERMINAL\b\s*-?", " "), 
    (r"\bPOINT\s+OF\s+SALE\s+(?:WITHDRAWAL|DEBIT)\b\s*-?", " "), 
    (r"\b(?:CRD|ACH)\s+TRAN(?:\s+PPD(?:\s+ID)?)?\b", " "), 
    (r"\bCO\s+ID\s+\w+\s+(?:WEB|PPD)\b.*", " "), # Remove CO ID... 
     
    # === 6) Misc tails === 
    (r"\b(?:INST|PAYPAL)\s+XFER\b", " "), 
    (r"\b(?:XFER|WEB)\s+ID\b.*", " "), 
    (r"\b(?:ELECTRONIC|EXTERNAL)\s+WITHDRAWAL\b", " "), 
    (r"\bWITHDRAWAL\s+DEBIT\s+CARD\b(?:\s+DEBIT)?", " "), 
    (r"\bO(?:F)?\s+SALE\s+DEBIT\s+L\d{3}\b.*", " "), 
    (r"\b(?:ITEM|OVERDRAFT)\s+FEE\s+FOR\s+ACTIVITY\b.*", " "), 
    (r"\b(?:GENESIS[-\s]*FS\s+CARD\s+PAYMENT)\b", " "), 
    (r"\bBILL\s+PAYMENT\b", " "), 
    (r"\b(?:US|WA)\s+CARD\s+PURCHASE\b", " "), 
    (r"-\s*MEMO=", " "), 
    (r"(?:USA|US)$", " "), # Remove USA or US at the end 
    (r"\s+FSP$", " "), 
    (r"\bL\d{3}\b", " "), # Handle L340

    # === 7) Phone numbers === 
    (r"\b(?:\d{3}-\d{3}-\d{4}|XXX-XXX-XXXX)\b", " "), # 800-555-1212 
    (r"\b\d{3}-\d{4}\b", " "), # 555-1212 
    (r"\b(?:\d{3}\s*){1,2}\d{3}\s*\d{3,4}\b", " "), # 800 555 1212 or 1 800 555 1212 
    (r"\b#?\s*\d{3}-\d{3}-\d{1,4}\s*(?:AM|PM)?\b", " "), 

    # === 8) URLs/domains === 
    (r"^\.COM\s+BILL\b.*", " "), 
    (r"\.COM$", " "), # Remove .COM at end of string

    # === 9) State abbreviations === 
    (STATE_REGEX, " "), # Remove standalone state codes 
    
    # === 10) Noise Words (from 1-grams) ===
    (NOISE_WORDS_REGEX, " "),

    # === 11) Final Tidy (Punctuation) === 
    (r"[|%_=;\\/]+", " "), # Remove misc separators 
    (r"[-]{2,}", " "), # Collapse multiple hyphens
    (r"^\s*-\s*|\s*-\s*$", " "), # Remove leading/trailing hyphens
]

In [9]:
REGEX_POST = [
    # --- Specific/Tricky Rules ---
    # This greedily finds the *last* instance of GODADDY.COM or GODADDY
    re.compile(r".*(GODADDY\.COM|GODADDY)\b.*"),
    
    # Rules for 'PAYPAL [MERCHANT] INTERNET PAYMENT' format
    re.compile(r"^PAYPAL\s+([A-Z\s0-9'.*-]+?)\s+(?:INTERNET\s+PAYMENT|COID.*)?.*$"),

    # Rules for '[MERCHANT] ... PAYPAL' format
    re.compile(r"^([A-Z\s0-9'.*-]+?)\s+PAYPAL.*$"),

    # === NEW PAYPAL RULES ===
    # Handle 'PAYPAL *... MERCHANT' (from INST XFER)
    re.compile(r"^PAYPAL\s*\*+\s*([A-Z\s0-9'.-]+).*"),
    # Handle 'PAYPAL : MERCHANT' (from INST XFER / ID:)
    re.compile(r"^PAYPAL\s*:\s*([A-Z\s0-9'.-]+).*"),

    # === NEW: High-Frequency Full Merchant Names (from 1-grams) ===
    re.compile(r"^(AMZN(?:\s*MKTP)?)\b.*"),
    re.compile(r"^(AMAZON(?:\.COM|\s+PRIME)?)\b.*"),
    re.compile(r"^(APPLE(?:\.COM)?)\b.*"),
    re.compile(r"^(CASH\s+APP)\b.*"),
    re.compile(r"^(WALMART(?:\s*SUPERCENTER)?|WM\s*SUPERCENTER|WAL-MART)\b.*"),
    re.compile(r"^(SAMS\s*CLUB)\b.*"),
    re.compile(r"^(DOORDASH)\b.*"),
    re.compile(r"^(MCDONALD'?S)\b.*"),
    re.compile(r"^(TARGET)\b.*"),
    re.compile(r"^(GOOGLE)\b.*"),
    re.compile(r"^(UBER(?:\s+EATS)?)\b.*"),
    re.compile(r"^(TACO\s+BELL)\b.*"),
    re.compile(r"^(STARBUCKS)\b.*"),
    re.compile(r"^(BURGER\s+KING)\b.*"),
    re.compile(r"^(PUBLIX)\b.*"),
    re.compile(r"^(CIRCLE\s+K)\b.*"),
    re.compile(r"^(AFTERPAY)\b.*"),
    re.compile(r"^(CHICK-FIL-A)\b.*"),
    re.compile(r"^(SUBWAY)\b.*"),
    re.compile(r"^(KROGER)\b.*"),
    re.compile(r"^(HOME\s+DEPOT)\b.*"),
    re.compile(r"^(DUNKIN)\b.*"),
    re.compile(r"^(LITTLE\s+CAESARS)\b.*"),
    re.compile(r"^(DOLLAR\s+GENERAL)\b.*"),
    re.compile(r"^(DOLLAR\s+TREE)\b.*"),
    re.compile(r"^(FAMILY\s+DOLLAR)\b.*"),
    re.compile(r"^(USPS)\b.*"),
    re.compile(r"^(BRIGIT)\b.*"),
    re.compile(r"^(INSTACART)\b.*"),
    re.compile(r"^(FRYS)\b.*"),
    re.compile(r"^(ROSS)\b.*"),
    re.compile(r"^(MICROSOFT)\b.*"),
    re.compile(r"^(ALDI)\b.*"),
    re.compile(r"^(COSTCO)\b.*"),
    re.compile(r"^(KFC)\b.*"),
    re.compile(r"^(SONIC)\b.*"),
    re.compile(r"^(LYFT)\b.*"),
    re.compile(r"^(HELPPAY)\b.*"),
    re.compile(r"^(DAIRY\s+QUEEN)\b.*"),
    re.compile(r"^(SAFEWAY)\b.*"),
    re.compile(r"^(WENDY'?S)\b.*"),
    re.compile(r"^(ETSY)\b.*"),
    re.compile(r"^(SHOPRITE)\b.*"),
    re.compile(r"^(LOWE'?S)\b.*"),
    re.compile(r"^(CHIPOTLE)\b.*"),
    re.compile(r"^(VONS)\b.*"),
    re.compile(r"^(FOOD\s+LION)\b.*"),

    # --- Standardized Prefix Rules ---
    # (Capture group is placed *after* the prefix)
    re.compile(r"^ACI\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^KING\s*#\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^ACE\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    
    # === 7-ELEVEN RULES ===
    re.compile(r"^(7(?:-ELEVEN|\s+11))\s*\*?#?.*"),
    
    re.compile(r"^ZG\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^YSI\s*\*?\s*([A-Z\s0-9'.-]+).*"),

    # === UPDATED RULE ORDER ===
    re.compile(r"^(DDBR)\s*\*?#?.*"), 
    
    re.compile(r"^DD\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^CCI\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^PY\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^ANC\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^J2\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^OSP\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^PL\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^RTI\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^FSP\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^PT\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^PP\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^CHR\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^USA\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^SP\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    
    # === TST RULES ===
    re.compile(r"^(?:POS\s+)?TST\s*\*?\s*([A-Z\s0-9'.-]+?)\s+[A-Z]{2,}\s+[A-Z]{2,}\s+ON\s*##.*"),
    re.compile(r"^(?:POS\s+)?TST\s*\*?\s*([A-Z\s0-9'.-]+?)\s+[A-Z]{2,}\s+ON\s*##.*"),
    re.compile(r"^(?:POS\s+)?TST\s*\*?\s*([A-Z\s0-9'.-]+).*"), 
    
    re.compile(r"^CKE\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^SQ\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^SP\+AFF\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^IC\s*\*?\s*([A-Z\s0-9'.-]+).*"), 
    re.compile(r"^SIE\s*\*?\s*([A-Z\s0-9'.-]+).*"),
    
    # Generic PAYPAL rule, catches PAYPAL*MERCHANT
    re.compile(r"^PAYPAL\s*\*?\s*([A-Z\s0-9'.-]+).*"), 

    # --- Specific '*' prefix rules (already correct) ---
    re.compile(r"^AMS\s*\*+\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^ZSK\s*\*+ZSK\s*\*+([A-Z\s0-9'.-]+).*"),
    re.compile(r"^ZSK\s*\*+\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^CCM\s*\*+\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^ZIP\.CO\s*\*+\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^XSOLLA\s*\*+\s*([A-Z\s0-9'.-]+).*"),
    re.compile(r"^NOR\s*\*+\s*([A-Z\s0-9'.-]+).*"),

    
    # --- Generic Fallback Rules ---
    # These rules try to find the merchant *before* a common delimiter.
    
    # Pattern for "MERCHANT NAME * JUNK"
    re.compile(r"^([A-Z\s'.-][A-Z\s0-9'.-]*?)\s*\*.*"),
    
    # Pattern for "MERCHANT NAME # JUNK"
    re.compile(r"^([A-Z\s'.-][A-Z\s0-9'.-]*?)\s*#.*"),
    
    # Pattern for "MERCHANT NAME - Exp"
    re.compile(r"^([A-Z\s'.-][A-Z\s0-9'.-]*?)\s+-\s+EXP.*"),
]

## 3. Apply Regex

In [10]:
%%time
# First pass
memos = df['memo'].astype(str).fillna('').str.upper()
memos = memos.str.replace(r"\u00A0", " ", regex=True)
memos = memos.str.replace(r"\s{2,}", " ", regex=True)
memos = memos.str.strip()

for pattern, repl in REGEX_PRE:
    memos = memos.str.replace(pattern, repl, regex=True)

memos = memos.str.replace(NOISE_WORDS_REGEX, " ", regex=True)
memos = memos.str.replace(r"\s{2,}", " ", regex=True)
memos = memos.str.replace(r"^[\s-]+|[\s-]+$", "", regex=True)
df['memo_pre'] = memos

CPU times: user 24.7 s, sys: 153 ms, total: 24.8 s
Wall time: 24.8 s


In [11]:
%%time
# Second pass
def apply_regex(memo):
    for pattern in REGEX_POST:
        match = pattern.match(memo)
        if match:
            return match.group(1).strip()
    return memo
df['memo_post'] = df['memo_pre'].apply(apply_regex)

CPU times: user 4.71 s, sys: 4.32 ms, total: 4.71 s
Wall time: 4.71 s


In [12]:
# df.to_csv('memos_P1.csv', index=False) # Save to CSV

In [13]:
# Manually check
# unique_df = df.drop_duplicates(subset='memo_post')
# result = (
#     unique_df[unique_df['memo_pre'].str.split().str.len() == 1]
#     .sort_values(by='memo_pre')[100:200].to_string()
# )
# print(result)

In [30]:
df.sample(10).sort_values(by='memo_post')

,memo,memo_pre,memo_post
178529,"IN *TNK VENTURES, LLC MESA AZ XXXXXX 0...","*TNK VENTURES, MESA","*TNK VENTURES, MESA"
522234,XXXXXX PURCHASE-PIN 07/05 22:23 7-ELEVEN WESTM...,22:23 7-ELEVEN WESTMINSTER 00MT,22:23 7-ELEVEN WESTMINSTER 00MT
60651,CASH APP*STATION 2B XXXXXXXXXX CA 12/17,CASH APP*STATION 2B,CASH APP
519966,XXXXXX PIN CHK PURCH SANTA FE FOODS CASTROVILL...,CHK SANTA FE FOODS CASTROVILLE,CHK SANTA FE FOODS CASTROVILLE
219358,PAYPAL DES:INST XFER ID:APPLE.COM BILL INDN:HI...,PAYPAL DES: :APPLE.COM BILL :HIT ELECTRONICS :...,DES
312874,PURCHASE AUTHORIZED ON 05/22 DIY HOME CENTER #...,DIY HOME CENTER #2 BIG BEAR LAKE S,DIY HOME CENTER
181534,JCPENNEY XXXX BROOKLYN N,JCPENNEY BROOKLYN N,JCPENNEY BROOKLYN N
283581,PURCHASE AUTHORIZED ON 03/07 JG'S OLD FASHIONE...,JG'S OLD FASHIONED DALLAS S,JG'S OLD FASHIONED DALLAS S
192612,LUCKY 7 DELI BRISTOL PA 07/25,LUCKY 7 DELI BRISTOL,LUCKY 7 DELI BRISTOL
250616,POS SIG 07/26 VISA #XXXX JACKSONVILLE VAN 4X4 ...,POS # JACKSONVILLE VAN 4X4 JACKSONVILLE,POS


In [15]:
merchants_clean = ['AMAZON', 'ALBERTSONS', 'ADOBE', 'SONIC', 'LIDL', 'AFTERPAY', 'ARBYS', 'ALDI', 'AUDIBLE', 'DOLLARTREE', 'LOWES', 'KROGER', 'DUNKIN', 'HP', 'PUBLIX', 'FRYS', 'SAFEWAY', 'DOORDASH', 'GOOGLE', 'WALMART', 'CHICK-FIL-A', 'SAMSCLUB', 'MICROSOFT',
             'UBER', 'ULTA', 'H-E-B', 'VONS', 'CMSVEND', 'INSTACART', 'LYFT', 'TJMAXX', 'PETSMART', 'THORNTONS', 'PAYPAL', 'MAVERIK', 'WENDYS', 'MARSHALLS', 'ALLSUPS', 'SUNPASS', 'QVC', 'PRIZEPICKS', 'DDBR']

merchants_punc = ["DENNY'S", 'WAL-MART', "LOWE'S", 'DOLLAR-GENERAL', '7-ELEVEN', "WENDY'S", "ZAXBY'S", 'FRYS-FOOD-DRG', "BUC-EE'S",
                 "BASHAS'"]

merchants_sites = ['AMAZON.COM', 'GODADDY.COM', 'CCBILL.COM']

merchant_cats = ['', 'OVERDRAFT', 'WITHDRAWAL']

multiples = [['AMAZON', 'AMAZON.COM', 'AMAZON PRIME'], ['LOWES', "LOWE'S"], ['BASKIN', 'DDBR']]

merchants = merchants_clean + merchants_punc + merchants_sites + merchant_cats

In [16]:
for memo in df[df['memo_post'].str.split().str.len() == 1]['memo_post'].value_counts().sort_values(ascending=False).index:
    if memo not in merchants:
        print(memo)

MCDONALD'S
TARGET
APPLE.COM
STARBUCKS
AMZN
SUBWAY
POS
USPS
BRIGIT
COSTCO
ROSS
APPLE
KFC
ETSY
SHOPRITE
CHIPOTLE
MCDONALDS
AMZNFREETIME
WINN-DIXIE
DES
SHIPT
GOFAN
BASKIN
RALPHS
BETMGM
SLICE
PETCO
CHEWY.COM
IHOP
INTUIT
STEAK-N-SHAKE
DILLONS
STATERBRO
*STARBUCKS
VANS
SKILLZ
MGM
FRG
TOLLWAY-AUTOREPLEN
.KOHLS.COM
PRICELN
BANFIELD-PET
RIOT
GAMESTOP
MCW
POPEYES
P
SAVEMART
VERIZONWRLSS
FOODMAXX
OCULUS
BASHAS''
MARINA
CANVA
RAINBOW
ABC
BELK
WALGREENS
GNC
MEIJER
CLAIRE'S
SEZZLE
FOOD4LESS
QFC
STAPLES
*EBAY
FIV
NYTIMES
AMZ
V
L
GOODWILL
TILLYS
*UBER
SHOPIFY
UPS
*MICROSOFT
IBI
ETT
OTT
BLUESKY
DROPBOX
POTBELLY
EL
JACK'S
WEGMANS
QUADPAY
ROSES
CHECKERS
JOURNEYS
LUCKY
EVI
RVT
ENMARKET
ABCMOUSE.COM
NORTON
FBPAY
EA
CRT
GERALD
HLLFRSH
*STEAM
HOME
EBAY
OPC
DRI
TLG
REI
LJS
NORDSTROM
SEDANOS
CRYPTO.COM
DEBIT
ZTL
EZPASS
NEWSSTAND
FH
PARKMOBILE
SOUTHWES
EVERYPLATE
UBR
E-Z
RGP
MOE'S
FRED-MEYER
PACSUN
M
PAR
TRTHFDR
EPC
TLF
EXPRESS
GLOSS
PCH
CMS
BUCKLE
PEET'S
CKO
GOFNDME
BOXYCHARM
ECS
LIQUOR
FACEBK
ARBY'S
HANNAFORD

In [17]:
df[df['memo_post'] == '']#.iloc[0]['memo']

,memo,memo_pre,memo_post
8876,ACH/DISCOVER,,
71408,CHECKCARD XXXX ARXXXX,,
74774,CHECKCARD XXXX CK XXXXXXX,,
82514,CHECKCARD XXXX MCC NC XXXXXXXXXXXXXXXXXXXXXXX,,
88157,"CHECKCARD XXXX SQ *""IT'S ALL PEACHY"" F Centenn...","SQ *""IT'S ALL PEACHY"" F CENTENNIAL",
93788,CHECKCARD XXXX XXXX XXXXXXXXXX TX XXXXXXXXXXXX...,,
93843,CHECKCARD XXXX XXXXXXXXXXXX,,
93846,CHECKCARD XXXX XXXXXXXXXXXXXX,,
98408,CK XXXXXXX,,
104841,Card,,


In [18]:
df[df['memo_post'] == '']

,memo,memo_pre,memo_post
8876,ACH/DISCOVER,,
71408,CHECKCARD XXXX ARXXXX,,
74774,CHECKCARD XXXX CK XXXXXXX,,
82514,CHECKCARD XXXX MCC NC XXXXXXXXXXXXXXXXXXXXXXX,,
88157,"CHECKCARD XXXX SQ *""IT'S ALL PEACHY"" F Centenn...","SQ *""IT'S ALL PEACHY"" F CENTENNIAL",
93788,CHECKCARD XXXX XXXX XXXXXXXXXX TX XXXXXXXXXXXX...,,
93843,CHECKCARD XXXX XXXXXXXXXXXX,,
93846,CHECKCARD XXXX XXXXXXXXXXXXXX,,
98408,CK XXXXXXX,,
104841,Card,,


In [19]:
regex_pattern = r"^[A-Z0-9\.]+\s*\*\s*[A-Z\s0-9'.-]+"

prefix_star_merchants = df[df['memo_pre'].str.contains(regex_pattern, na=False)]
prefix_star_merchants

,memo,memo_pre,memo_post
2212,13FLDENVER* 13THFL HTTPSWWW.13TH CO,13FLDENVER* 13THFL .13TH,13FLDENVER* 13THFL .13TH
2331,1PASSWORD* TRIAL OVER TORONTO ON 01/07,1PASSWORD* TRIAL OVER TORONTO ON,1PASSWORD* TRIAL OVER TORONTO ON
3607,2CHECKO*KILOHEA Alpharetta GA USA,2CHECKO*KILOHEA ALPHARETTA,2CHECKO*KILOHEA ALPHARETTA
3608,2CO.COM*slideupli XXXXXXXXXX 04/03,2CO.COM*SLIDEUPLI,2CO.COM*SLIDEUPLI
3609,2COCOM*BITDEFENDER.COM,2COCOM*BITDEFENDER,2COCOM*BITDEFENDER
...,...,...,...
528725,www.Playgr* Bidiboo.Co,.PLAYGR* BIDIBOO.CO,.PLAYGR
528726,www.Playgr* Littlemiss,.PLAYGR* LITTLEMISS,.PLAYGR
528732,www.Styles* Luv N Hair,.STYLES* LUV N HAIR,.STYLES
528733,www.Stylese* West Brim,.STYLESE* WEST BRIM,.STYLESE


In [20]:
print(prefix_star_merchants['memo_pre'].str.split('*').sample(100).sort_values().to_string())

436391        [4TE, CULLIGAN OF MARYSVILL NORTH CANTONDATE]
32245                      [AMAZON , LK5AT41 AMZN.COM BILL]
63875                                  [AMAZON , YZ3AT6QZ3]
332309                [AMAZON.COM, 1N3PY8J AMZN.COM BILL S]
228394    [AMAZON.COM, 1Q2U55F AMZN.COM B JENNIFER E COM...
361887                [AMAZON.COM, 1U5FK5U AMZN.COM BILL S]
228437    [AMAZON.COM, 1V AMZN.COM B JENNIFER E COMSTOCK...
14635              [AMAZON.COM, 2C44M8TK1 SEATT LE P4IXUUR]
65021                  [AMAZON.COM, 2G6TF6 AMZN.COM BILLWA]
316889                [AMAZON.COM, 2R3C14Y AMZN.COM BILL S]
321127                [AMAZON.COM, 2X3CO1S AMZN.COM BILL S]
15129             [AMAZON.COM, 2Y1PR5EF2 SEATT LE CLXSP2B4]
36837                 [AMAZON.COM, CO7JU4YA3 AMZN.COM BILL]
248293    [AMAZON.COM, CY719 AMAZON.COM SEATTLE US NBR: ...
324491                [AMAZON.COM, F16TY3R AMZN.COM BILL S]
65781            [AMAZON.COM, FI2DP4Y93 AM AMZN.COM BILLWA]
369933                [AMAZON.COM, H83U0

# Phase 2: Extract & Analyze N-Grams

In [21]:
# print(df[df['memo_post'].str.len() < 4]['memo_post'].to_string())

In [22]:
def top_ngrams(corpus: pd.Series, n_gram_range: tuple, top_n: int = 200):
    print(f"Analyzing {n_gram_range} n-grams...")
    vec = CountVectorizer(
        ngram_range=n_gram_range,
        stop_words='english',
        max_features=None  # We want to count all n-grams first
    ).fit(corpus)
    
    # Get the counts
    bag_of_words = vec.transform(corpus)
    
    # Sum the counts for each n-gram
    sum_words = bag_of_words.sum(axis=0)
    
    # Map n-grams to their frequencies
    words_freq = [
        (word, sum_words[0, idx]) 
        for word, idx in vec.vocabulary_.items()
    ]
    
    # Sort by frequency (descending)
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:top_n]

In [23]:
%%time
corpus = df['memo_post'].fillna('')
print(f"Analyzing {len(corpus)} cleaned memos...")
# Get the top 200 of each n-gram type
top_1grams = top_ngrams(corpus, n_gram_range=(1, 1), top_n=100)
top_2grams = top_ngrams(corpus, n_gram_range=(2, 2), top_n=100)
top_3grams = top_ngrams(corpus, n_gram_range=(3, 3), top_n=100)
print(f"--- N-gram Analysis Complete ---")

Analyzing 528766 cleaned memos...
Analyzing (1, 1) n-grams...
Analyzing (2, 2) n-grams...
Analyzing (3, 3) n-grams...
--- N-gram Analysis Complete ---
CPU times: user 10.7 s, sys: 4.38 ms, total: 10.7 s
Wall time: 10.7 s


In [24]:
top_1grams
ngrams_1 = []
for ngram, value in top_1grams:
    if ngram.upper() not in NOISE_WORDS:
        ngrams_1 += [ngram]

In [25]:
top_1grams

[('com', 43438),
 ('xxx', 34747),
 ('amazon', 32484),
 ('amzn', 22493),
 ('cash', 21287),
 ('app', 19346),
 ('mktp', 18994),
 ('mart', 14573),
 ('wal', 13290),
 ('pos', 13090),
 ('doordash', 12163),
 ('purchase', 10633),
 ('market', 8860),
 ('mcdonald', 8857),
 ('debit', 8140),
 ('dollar', 8070),
 ('apple', 8000),
 ('san', 6658),
 ('google', 6644),
 ('food', 6251),
 ('target', 6034),
 ('wm', 5818),
 ('pizza', 5166),
 ('taco', 4719),
 ('city', 4691),
 ('prime', 4169),
 ('cafe', 3905),
 ('starbucks', 3855),
 ('king', 3853),
 ('st', 3678),
 ('burger', 3500),
 ('mobile', 3455),
 ('el', 3401),
 ('shop', 3304),
 ('circle', 3293),
 ('superc', 3280),
 ('publix', 3279),
 ('00', 3278),
 ('general', 3180),
 ('new', 3143),
 ('liquor', 3101),
 ('uber', 3059),
 ('afterpay', 3000),
 ('sup', 2895),
 ('bar', 2810),
 ('fort', 2768),
 ('miami', 2744),
 ('ebay', 2731),
 ('vending', 2700),
 ('bell', 2659),
 ('express', 2618),
 ('subway', 2594),
 ('15', 2592),
 ('troy', 2577),
 ('chick', 2568),
 ('valley', 

In [26]:
ngrams_1

['com',
 'xxx',
 'amazon',
 'amzn',
 'cash',
 'app',
 'mktp',
 'mart',
 'wal',
 'pos',
 'doordash',
 'purchase',
 'market',
 'mcdonald',
 'debit',
 'dollar',
 'apple',
 'san',
 'google',
 'food',
 'target',
 'wm',
 'pizza',
 'taco',
 'city',
 'prime',
 'cafe',
 'starbucks',
 'king',
 'st',
 'burger',
 'mobile',
 'el',
 'shop',
 'circle',
 'superc',
 'publix',
 '00',
 'general',
 'new',
 'liquor',
 'uber',
 'afterpay',
 'sup',
 'bar',
 'fort',
 'miami',
 'ebay',
 'vending',
 'bell',
 'express',
 'subway',
 '15',
 'troy',
 'chick',
 'valley',
 'kroger',
 '20',
 'home',
 'supercenter',
 'house',
 'fil',
 'dunkin',
 'lake',
 'seattle',
 '365',
 'little',
 'diego',
 'grill',
 'family',
 '16',
 'west',
 'phoenix',
 'depot',
 'beach',
 'tr',
 'santa',
 'usps',
 'coffee',
 'chicago',
 'brigit',
 'mexican',
 '10',
 'las',
 'wine',
 'instacart',
 'york',
 'club',
 'los',
 'park',
 'walmart',
 'houston',
 'frys',
 'orlando',
 'atlanta',
 'north',
 'beauty',
 'ross',
 'microsoft',
 'aldi']

In [27]:
top_2grams

[('amazon com', 24188),
 ('amzn mktp', 18946),
 ('cash app', 18686),
 ('xxx xxx', 13307),
 ('wal mart', 11511),
 ('apple com', 6144),
 ('amazon prime', 3851),
 ('wm superc', 3278),
 ('superc wal', 3277),
 ('dollar general', 3050),
 ('mobile purchase', 2991),
 ('mart sup', 2692),
 ('com xxx', 2457),
 ('wm supercenter', 2408),
 ('taco bell', 2408),
 ('chick fil', 2362),
 ('365 market', 2205),
 ('san diego', 2168),
 ('debit purchase', 1966),
 ('dollar tr', 1835),
 ('burger king', 1828),
 ('little caesars', 1529),
 ('home depot', 1482),
 ('new york', 1467),
 ('las vegas', 1384),
 ('family dollar', 1306),
 ('sams club', 1092),
 ('market 432', 1031),
 ('los angeles', 1021),
 ('hunt valley', 999),
 ('san antonio', 983),
 ('uber eats', 947),
 ('food lion', 946),
 ('amzn com', 891),
 ('wal wal', 880),
 ('stop shop', 876),
 ('dairy queen', 791),
 ('jack box', 774),
 ('dollar tree', 770),
 ('com seattle', 746),
 ('winco foods', 733),
 ('carls jr', 728),
 ('purchase sign', 725),
 ('sign based', 72

In [28]:
top_3grams

[('wm superc wal', 3277),
 ('superc wal mart', 2766),
 ('wal mart sup', 2692),
 ('com xxx xxx', 1788),
 ('365 market 432', 1031),
 ('wal wal mart', 875),
 ('amazon com seattle', 742),
 ('mobile purchase sign', 725),
 ('purchase sign based', 725),
 ('apple com xxx', 661),
 ('cash 20 00', 640),
 ('market work renton', 566),
 ('snack soda vending', 557),
 ('market xxx xxx', 507),
 ('dollar ge dg', 502),
 ('365 market xxx', 497),
 ('xxx xxx troy', 492),
 ('365 market 43', 469),
 ('market 43 troy', 466),
 ('market 432 32', 462),
 ('432 32 troy', 455),
 ('pos amazon com', 453),
 ('bath body works', 450),
 ('market 432 329', 441),
 ('432 329 troy', 439),
 ('salt lake cit', 402),
 ('cash 40 00', 400),
 ('domino xxx xxx', 380),
 ('purchase amazon com', 379),
 ('help hbomax com', 372),
 ('klover app boost', 363),
 ('merchant issued target', 363),
 ('fresh market coffe', 339),
 ('market coffe waxahachie', 339),
 ('point sale debitl340', 335),
 ('xxx xxx 15', 316),
 ('hbomax com httpshbomax', 312)

In [29]:
# Use 1 grams to find prefixes